# Algorithm & Metric Deep Dive - Making appropriate choices for your problem

NOOA template sections
## Overview 

### Prerequisites
xx

### Learning Outcomes 
xx

## Tutorial 
xx

## Excercise 
xx

## Examples of use
xx
 
## Next steps
xx

## Dataset Info
xx

## References
xx


## Problem 3 - Radiation Emulation (Regression)